# Libraries

In [1]:
# LIBRARIES #

import numpy as np

from scipy import optimize
from scipy.interpolate import CubicSpline, interp1d

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

from skimage import filters

import SimpleITK as sitk

from Gafchromic import GafchromicFilms


output_notebook()

Loading BokehJS ...

In [2]:
# FONCTIONS:


# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleRGBArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor,:]\
                        .reshape((newsizey, subfactor, newsizex, subfactor, 3)).mean(3).mean(1)
    
    return newarray



# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    
    
# Converts the RGB image to RGBA for display
#   
# @params:
#  array: image RGB array
#  sizex: x size of the image
#  sizey: y size of the image
def convertToRGBA(array, sizex, sizey):

    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return rgba




# Reads all images and returns the median image
#   
# @params:
#  array: image RGB array
#  sizex: x size of the image
#  sizey: y size of the image
def medianImage(path, filename, firstNb, fileExtension, nbOfImgs):

    img = sitk.ReadImage(path+filename+str(firstNb)+fileExtension)

    size = (sitk.GetArrayFromImage(img).shape[0], 
            sitk.GetArrayFromImage(img).shape[1], 
            sitk.GetArrayFromImage(img).shape[2], 
            nbOfImgs)
    
    imgs = np.zeros(size)
    
    for i in range(nbOfImgs):
        img = sitk.ReadImage(path+filename+str(firstNb+i)+fileExtension)
        imgs[:,:,:,i] = sitk.GetArrayFromImage(img)

    medianImg = np.median(imgs, axis=3)

    return medianImg




# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    
    
    
# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def dispImgAndProf(img, col, line, plotwidth=450, title='dose image 1', colorprofile='firebrick'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img.shape[0]/img.shape[1]), 
                title=title, toolbar_location="above")
    p1.image(image=[img], x=0, y=0, dw=img.shape[1], dh=img.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img[line,:]), 1)
    p3.line(x3, img[line,:], line_width=2, line_color=colorprofile, legend_label=title)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img[:,col]), 1)
    p4.line(x4, img[:, col], line_width=3, line_color=colorprofile, legend_label=title)

    grid = gridplot([[p3, p4]])

    show(p1)
    show(grid)
    

# Computes coefficients

In [3]:
# READS THE IMAGES:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers


# Inputs:

m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/11 - correction laterale/scans_V2/'
m_firstNb = 1
m_nbOfFiles = 3

m_file40mm = "Scan_40mm_00"
m_file80mm = "Scan_80mm_00"
m_file120mm = "Scan_120mm_00"
m_file160mm = "Scan_160mm_00"
m_file200mm = "Scan_200mm_00"
m_file240mm = "Scan_240mm_00"
m_file280mm = "Scan_280mm_00"

m_fileExtension = ".tif"

m_dimViewer = 600
m_subfactor = 3

visu = False


# Reading the images:
img40mm = medianImage(m_path, m_file40mm, m_firstNb, m_fileExtension, m_nbOfFiles)
img40mm = subSampleRGBArray(img40mm, m_subfactor)

img80mm = medianImage(m_path, m_file80mm, m_firstNb, m_fileExtension, m_nbOfFiles)
img80mm = subSampleRGBArray(img80mm, m_subfactor)

img120mm = medianImage(m_path, m_file120mm, m_firstNb, m_fileExtension, m_nbOfFiles)
img120mm = subSampleRGBArray(img120mm, m_subfactor)

img160mm = medianImage(m_path, m_file160mm, m_firstNb, m_fileExtension, m_nbOfFiles)
img160mm = subSampleRGBArray(img160mm, m_subfactor)

img200mm = medianImage(m_path, m_file200mm, m_firstNb, m_fileExtension, m_nbOfFiles)
img200mm = subSampleRGBArray(img200mm, m_subfactor)

img240mm = medianImage(m_path, m_file240mm, m_firstNb, m_fileExtension, m_nbOfFiles)
img240mm = subSampleRGBArray(img240mm, m_subfactor)

img280mm = medianImage(m_path, m_file280mm, m_firstNb, m_fileExtension, m_nbOfFiles)
img280mm = subSampleRGBArray(img280mm, m_subfactor)


# rectangles for processing:
rect40mm = []
rect40mm.append([255,65,285,105])
rect40mm.append([255,135,285,175])
rect40mm.append([255,205,285,245])
rect40mm.append([255,275,285,315])

rect80mm = []
rect80mm.append([215,65,245,105])
rect80mm.append([215,135,245,175])
rect80mm.append([215,205,245,245])
rect80mm.append([215,275,245,315])

rect120mm = []
rect120mm.append([175,65,205,105])
rect120mm.append([175,135,205,175])
rect120mm.append([175,205,205,245])
rect120mm.append([175,275,205,315])

rect160mm = []
rect160mm.append([135,65,165,105])
rect160mm.append([135,135,165,175])
rect160mm.append([135,205,165,245])
rect160mm.append([135,275,165,315])

rect200mm = []
rect200mm.append([98,65,128,105])
rect200mm.append([98,135,128,175])
rect200mm.append([98,205,128,245])
rect200mm.append([98,275,128,315])

rect240mm = []
rect240mm.append([55,65,85,105])
rect240mm.append([55,135,85,175])
rect240mm.append([55,205,85,245])
rect240mm.append([55,275,85,315])

rect280mm = []
rect280mm.append([15,65,45,105])
rect280mm.append([15,135,45,175])
rect280mm.append([15,205,45,245])
rect280mm.append([15,275,45,315])


# Viusalisation:
if visu:
    m_dimMosaiq = 300

    p1 = figure(plot_width=m_dimMosaiq, plot_height=int(m_dimMosaiq*img40mm.shape[0]/img40mm.shape[1]), 
               title='40mm', toolbar_location="below")
    p1.image(image=[img40mm[:,:,0]], x=0, y=0, dw=img40mm.shape[1], dh=img40mm.shape[0], palette="Greys256")
    for i in range(len(rect40mm)):
        p1.rect(x=int(rect40mm[i][0]+(rect40mm[i][2]-rect40mm[i][0])/2), 
                   y=int(rect40mm[i][1]+(rect40mm[i][3]-rect40mm[i][1])/2), 
                   width=rect40mm[i][2]-rect40mm[i][0],
                   height=rect40mm[i][3]-rect40mm[i][1], 
                   fill_color="yellow", 
                   alpha=0.5)

    p2 = figure(plot_width=m_dimMosaiq, plot_height=int(m_dimMosaiq*img80mm.shape[0]/img80mm.shape[1]), 
               title='80mm', toolbar_location="below")
    p2.image(image=[img80mm[:,:,0]], x=0, y=0, dw=img80mm.shape[1], dh=img80mm.shape[0], palette="Greys256")
    for i in range(len(rect80mm)):
        p2.rect(x=int(rect80mm[i][0]+(rect80mm[i][2]-rect80mm[i][0])/2), 
                   y=int(rect80mm[i][1]+(rect80mm[i][3]-rect80mm[i][1])/2), 
                   width=rect80mm[i][2]-rect80mm[i][0],
                   height=rect80mm[i][3]-rect80mm[i][1], 
                   fill_color="yellow", 
                   alpha=0.5)

    p3 = figure(plot_width=m_dimMosaiq, plot_height=int(m_dimMosaiq*img120mm.shape[0]/img120mm.shape[1]), 
               title='120mm', toolbar_location="below")
    p3.image(image=[img120mm[:,:,0]], x=0, y=0, dw=img120mm.shape[1], dh=img120mm.shape[0], palette="Greys256")
    for i in range(len(rect120mm)):
        p3.rect(x=int(rect120mm[i][0]+(rect120mm[i][2]-rect120mm[i][0])/2), 
                   y=int(rect120mm[i][1]+(rect120mm[i][3]-rect120mm[i][1])/2), 
                   width=rect120mm[i][2]-rect120mm[i][0],
                   height=rect120mm[i][3]-rect120mm[i][1], 
                   fill_color="yellow", 
                   alpha=0.5)

    p4 = figure(plot_width=m_dimMosaiq, plot_height=int(m_dimMosaiq*img160mm.shape[0]/img160mm.shape[1]), 
               title='160mm', toolbar_location="below")
    p4.image(image=[img160mm[:,:,0]], x=0, y=0, dw=img160mm.shape[1], dh=img160mm.shape[0], palette="Greys256")
    for i in range(len(rect160mm)):
        p4.rect(x=int(rect160mm[i][0]+(rect160mm[i][2]-rect160mm[i][0])/2), 
                   y=int(rect160mm[i][1]+(rect160mm[i][3]-rect160mm[i][1])/2), 
                   width=rect160mm[i][2]-rect160mm[i][0],
                   height=rect160mm[i][3]-rect160mm[i][1], 
                   fill_color="yellow", 
                   alpha=0.5)

    p5 = figure(plot_width=m_dimMosaiq, plot_height=int(m_dimMosaiq*img200mm.shape[0]/img200mm.shape[1]), 
               title='200mm', toolbar_location="below")
    p5.image(image=[img200mm[:,:,0]], x=0, y=0, dw=img200mm.shape[1], dh=img200mm.shape[0], palette="Greys256")
    for i in range(len(rect200mm)):
        p5.rect(x=int(rect200mm[i][0]+(rect200mm[i][2]-rect200mm[i][0])/2), 
                   y=int(rect200mm[i][1]+(rect200mm[i][3]-rect200mm[i][1])/2), 
                   width=rect200mm[i][2]-rect200mm[i][0],
                   height=rect200mm[i][3]-rect200mm[i][1], 
                   fill_color="yellow", 
                   alpha=0.5)

    p6 = figure(plot_width=m_dimMosaiq, plot_height=int(m_dimMosaiq*img240mm.shape[0]/img240mm.shape[1]), 
               title='240mm', toolbar_location="below")
    p6.image(image=[img240mm[:,:,0]], x=0, y=0, dw=img240mm.shape[1], dh=img240mm.shape[0], palette="Greys256")
    for i in range(len(rect240mm)):
        p6.rect(x=int(rect240mm[i][0]+(rect240mm[i][2]-rect240mm[i][0])/2), 
                   y=int(rect240mm[i][1]+(rect240mm[i][3]-rect240mm[i][1])/2), 
                   width=rect240mm[i][2]-rect240mm[i][0],
                   height=rect240mm[i][3]-rect240mm[i][1], 
                   fill_color="yellow", 
                   alpha=0.5)

    p7 = figure(plot_width=m_dimMosaiq, plot_height=int(m_dimMosaiq*img280mm.shape[0]/img280mm.shape[1]), 
               title='280mm', toolbar_location="below")
    p7.image(image=[img280mm[:,:,0]], x=0, y=0, dw=img280mm.shape[1], dh=img280mm.shape[0], palette="Greys256")
    for i in range(len(rect280mm)):
        p7.rect(x=int(rect280mm[i][0]+(rect280mm[i][2]-rect280mm[i][0])/2), 
                   y=int(rect280mm[i][1]+(rect280mm[i][3]-rect280mm[i][1])/2), 
                   width=rect280mm[i][2]-rect280mm[i][0],
                   height=rect280mm[i][3]-rect280mm[i][1], 
                   fill_color="yellow", 
                   alpha=0.5)

    grid = gridplot([[p1,p2,p3], [p4,p5,p6], [p7]])
    show(grid)

print('   >>> iMAGES rEAD !')

   >>> iMAGES rEAD !


In [4]:
# CORRECTION FACTORS DETERMINATION
#   Correction Factors calculated in Lewis et al. Med Phys january 2015

visu1 = False
visu2 = False

def func(x, a, b):
    return a + b*x


# reading ref. image (center=160mm):
centerValues_r = []
centerValues_g = []
centerValues_b = []
for i in range(len(rect160mm)):
    centerValues_r.append(np.mean(img160mm[rect160mm[i][1]:rect160mm[i][3], 
                                           rect160mm[i][0]:rect160mm[i][2], 0]))
    centerValues_g.append(np.mean(img160mm[rect160mm[i][1]:rect160mm[i][3], 
                                           rect160mm[i][0]:rect160mm[i][2], 1]))
    centerValues_b.append(np.mean(img160mm[rect160mm[i][1]:rect160mm[i][3], 
                                           rect160mm[i][0]:rect160mm[i][2], 2]))

    
# parameters table:
Ar = []
Br = []
Ag = []
Bg = []
Ab = []
Bb = []


# reading left image (40mm):
values40mm_r = []
values40mm_g = []
values40mm_b = []

for i in range(len(rect40mm)):
    values40mm_r.append(np.mean(img40mm[rect40mm[i][1]:rect40mm[i][3], 
                                           rect40mm[i][0]:rect40mm[i][2], 0]))
    values40mm_g.append(np.mean(img40mm[rect40mm[i][1]:rect40mm[i][3], 
                                           rect40mm[i][0]:rect40mm[i][2], 1]))
    values40mm_b.append(np.mean(img40mm[rect40mm[i][1]:rect40mm[i][3], 
                                           rect40mm[i][0]:rect40mm[i][2], 2]))

params, cov = optimize.curve_fit(func, values40mm_r, centerValues_r)
Ar.append(params[0])
Br.append(params[1])

params, cov = optimize.curve_fit(func, values40mm_g, centerValues_g)
Ag.append(params[0])
Bg.append(params[1])

params, cov = optimize.curve_fit(func, values40mm_b, centerValues_b)
Ab.append(params[0])
Bb.append(params[1])
    
    
# reading next image (80mm):
values80mm_r = []
values80mm_g = []
values80mm_b = []
for i in range(len(rect80mm)):
    values80mm_r.append(np.mean(img80mm[rect80mm[i][1]:rect80mm[i][3], 
                                           rect80mm[i][0]:rect80mm[i][2], 0]))
    values80mm_g.append(np.mean(img80mm[rect80mm[i][1]:rect80mm[i][3], 
                                           rect80mm[i][0]:rect80mm[i][2], 1]))
    values80mm_b.append(np.mean(img80mm[rect80mm[i][1]:rect80mm[i][3], 
                                           rect80mm[i][0]:rect80mm[i][2], 2]))

params, cov = optimize.curve_fit(func, values80mm_r, centerValues_r)
Ar.append(params[0])
Br.append(params[1])

params, cov = optimize.curve_fit(func, values80mm_g, centerValues_g)
Ag.append(params[0])
Bg.append(params[1])

params, cov = optimize.curve_fit(func, values80mm_b, centerValues_b)
Ab.append(params[0])
Bb.append(params[1])

    
# reading next image (120mm):
values120mm_r = []
values120mm_g = []
values120mm_b = []
for i in range(len(rect120mm)):
    values120mm_r.append(np.mean(img120mm[rect120mm[i][1]:rect120mm[i][3], 
                                           rect120mm[i][0]:rect120mm[i][2], 0]))
    values120mm_g.append(np.mean(img120mm[rect120mm[i][1]:rect120mm[i][3], 
                                           rect120mm[i][0]:rect120mm[i][2], 1]))
    values120mm_b.append(np.mean(img120mm[rect120mm[i][1]:rect120mm[i][3], 
                                           rect120mm[i][0]:rect120mm[i][2], 2]))

params, cov = optimize.curve_fit(func, values120mm_r, centerValues_r)
Ar.append(params[0])
Br.append(params[1])

params, cov = optimize.curve_fit(func, values120mm_g, centerValues_g)
Ag.append(params[0])
Bg.append(params[1])

params, cov = optimize.curve_fit(func, values120mm_b, centerValues_b)
Ab.append(params[0])
Bb.append(params[1])

    
# values for reference image (160mm):
Ar.append(0)
Br.append(1)

Ag.append(0)
Bg.append(1)

Ab.append(0)
Bb.append(1)


# reading next image (200mm):
values200mm_r = []
values200mm_g = []
values200mm_b = []
for i in range(len(rect200mm)):
    values200mm_r.append(np.mean(img200mm[rect200mm[i][1]:rect200mm[i][3], 
                                           rect200mm[i][0]:rect200mm[i][2], 0]))
    values200mm_g.append(np.mean(img200mm[rect200mm[i][1]:rect200mm[i][3], 
                                           rect200mm[i][0]:rect200mm[i][2], 1]))
    values200mm_b.append(np.mean(img200mm[rect200mm[i][1]:rect200mm[i][3], 
                                           rect200mm[i][0]:rect200mm[i][2], 2]))

params, cov = optimize.curve_fit(func, values200mm_r, centerValues_r)
Ar.append(params[0])
Br.append(params[1])

params, cov = optimize.curve_fit(func, values200mm_g, centerValues_g)
Ag.append(params[0])
Bg.append(params[1])

params, cov = optimize.curve_fit(func, values200mm_b, centerValues_b)
Ab.append(params[0])
Bb.append(params[1])


# reading next image (240mm):
values240mm_r = []
values240mm_g = []
values240mm_b = []
for i in range(len(rect240mm)):
    values240mm_r.append(np.mean(img240mm[rect240mm[i][1]:rect240mm[i][3], 
                                           rect240mm[i][0]:rect240mm[i][2], 0]))
    values240mm_g.append(np.mean(img240mm[rect240mm[i][1]:rect240mm[i][3], 
                                           rect240mm[i][0]:rect240mm[i][2], 1]))
    values240mm_b.append(np.mean(img240mm[rect240mm[i][1]:rect240mm[i][3], 
                                           rect240mm[i][0]:rect240mm[i][2], 2]))

params, cov = optimize.curve_fit(func, values240mm_r, centerValues_r)
Ar.append(params[0])
Br.append(params[1])

params, cov = optimize.curve_fit(func, values240mm_g, centerValues_g)
Ag.append(params[0])
Bg.append(params[1])

params, cov = optimize.curve_fit(func, values240mm_b, centerValues_b)
Ab.append(params[0])
Bb.append(params[1])


# reading right image (280mm):
values280mm_r = []
values280mm_g = []
values280mm_b = []
for i in range(len(rect280mm)):
    values280mm_r.append(np.mean(img280mm[rect280mm[i][1]:rect280mm[i][3], 
                                           rect280mm[i][0]:rect280mm[i][2], 0]))
    values280mm_g.append(np.mean(img280mm[rect280mm[i][1]:rect280mm[i][3], 
                                           rect280mm[i][0]:rect280mm[i][2], 1]))
    values280mm_b.append(np.mean(img280mm[rect280mm[i][1]:rect280mm[i][3], 
                                           rect280mm[i][0]:rect280mm[i][2], 2]))

params, cov = optimize.curve_fit(func, values280mm_r, centerValues_r)
Ar.append(params[0])
Br.append(params[1])

params, cov = optimize.curve_fit(func, values280mm_g, centerValues_g)
Ag.append(params[0])
Bg.append(params[1])

params, cov = optimize.curve_fit(func, values280mm_b, centerValues_b)
Ab.append(params[0])
Bb.append(params[1])
    
        
    
# data visualization:
signSize = 5
alpha = 0.5 

if visu1:
    p1 = figure(plot_width=500, plot_height=400, title="Red values 40mm", toolbar_location="above",
               x_axis_label='pixel value at L', y_axis_label='pixel value at center')
    p1.circle(values40mm_r, centerValues_r, size=signSize, color='firebrick', alpha=alpha)
    x1 = np.array([7000.0, 50000.0])
    y1 = func(x1, Ar[0], Br[0])
    p1.line(x1, y1, line_width=1, line_color='firebrick', line_dash='dotted')
    
    p2 = figure(plot_width=400, plot_height=300, title="Green values 40mm", toolbar_location="above",
               x_axis_label='pixel value at L', y_axis_label='pixel value at center')
    p2.circle(values40mm_g, centerValues_g, size=signSize, color='darkgreen', alpha=alpha)
    y1 = func(x1, Ag[0], Bg[0])
    p2.line(x1, y1, line_width=1, line_color='darkgreen', line_dash='dotted')
    
    p3 = figure(plot_width=400, plot_height=300, title="Blue values 40mm", toolbar_location="above",
               x_axis_label='pixel value at L', y_axis_label='pixel value at center')
    p3.circle(values40mm_b, centerValues_b, size=signSize, color='darkblue', alpha=alpha)
    y1 = func(x1, Ab[0], Bb[0])
    p3.line(x1, y1, line_width=1, line_color='darkblue', line_dash='dotted')

    grid = gridplot([[p1], [p2,p3]])
    show(grid)


if visu2:
    x = np.array([-120, -80, -40, 0, 40, 80, 120])

    p1 = figure(plot_width=700, plot_height=500, title="A values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p1.line(x, Ar, line_width=2, color='firebrick')
    p1.line(x, Ag, line_width=2, color='darkgreen')
    p1.line(x, Ab, line_width=2, color='darkblue')
    
    p2 = figure(plot_width=700, plot_height=500, title="B values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient B')
    p2.line(x, Br, line_width=2, color='firebrick')
    p2.line(x, Bg, line_width=2, color='darkgreen')
    p2.line(x, Bb, line_width=2, color='darkblue')

    grid = gridplot([[p1], [p2]])
    show(grid)

print('   >>> Coefficients Calculated !')

   >>> Coefficients Calculated !


In [19]:
# CORRECTION MODELS FITTING: 


# To visualize results:
visu = False


# Interpolation functions:
pos = np.array([-120, -80, -40, 0, 40, 80, 120])



fAr_lin = interp1d(pos, Ar, kind='linear')  # kind = cubic, linear, ...
fAg_lin = interp1d(pos, Ag, kind='linear')
fAb_lin = interp1d(pos, Ab, kind='linear')

fBr_lin = interp1d(pos, Br, kind='linear')
fBg_lin = interp1d(pos, Bg, kind='linear')
fBb_lin = interp1d(pos, Bb, kind='linear')


if visu:
    x = np.array([-120, -80, -40, 0, 40, 80, 120])
    xnew = np.arange(-120,120)
    
    p1 = figure(plot_width=300, plot_height=300, title="Ar values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p1.circle(x, Ar, size=5, color='firebrick', alpha=0.5)
    p1.line(xnew, fAr_lin(xnew), line_width=1, color='firebrick', line_dash='dashed')

    p2 = figure(plot_width=300, plot_height=300, title="Ag values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p2.circle(x, Ag, size=5, color='darkgreen', alpha=0.5)
    p2.line(xnew, fAg_lin(xnew), line_width=1, color='darkgreen', line_dash='dashed')

    p3 = figure(plot_width=300, plot_height=300, title="Ab values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p3.circle(x, Ab, size=5, color='darkblue', alpha=0.5)
    p3.line(xnew, fAb_lin(xnew), line_width=1, color='darkblue', line_dash='dashed')

#     p1.line(x, Ag, line_width=2, color='darkgreen')
#     p1.line(x, Ab, line_width=2, color='darkblue')
    
    p4 = figure(plot_width=300, plot_height=300, title="Br values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p4.circle(x, Br, size=5, color='firebrick', alpha=0.5)
    p4.line(xnew, fBr_lin(xnew), line_width=1, color='firebrick', line_dash='dashed')

    p5 = figure(plot_width=300, plot_height=300, title="Bg values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p5.circle(x, Bg, size=5, color='darkgreen', alpha=0.5)
    p5.line(xnew, fBg_lin(xnew), line_width=1, color='darkgreen', line_dash='dashed')

    p6 = figure(plot_width=300, plot_height=300, title="Bb values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p6.circle(x, Bb, size=5, color='darkblue', alpha=0.5)
    p6.line(xnew, fBb_lin(xnew), line_width=1, color='darkblue', line_dash='dashed')

    grid1 = gridplot([[p1,p2,p3],[p4,p5,p6]])
    show(grid1)

print("   >>> mODELS dONE!")

   >>> mODELS dONE!


# Images Correction:

In [33]:
# IMAGE CORRECTION AND CONVERSION TO DOSE:


# Variables:
path = 'testRBGB/'
nbOfFiles = 1
firstNb = 1
filesName = "testRBGB_00"
fileExtension = ".tif"
splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_data.txt'
pixsize = 1

# Reading the file:
try:
    g = GafchromicFilms(path+filesName, firstNb, nbOfFiles, fileExtension=fileExtension)
    g.subSampleDataArray(10)
    doseImg_noCorr = g.convertToDose_cubicSplineFit(splineFile, 1500)
    array = g.getArray()

    # correcting lateral response
    corrArray = np.zeros(array.shape)
    for i in range(array.shape[1]):
        posx = (i-(array.shape[1]/2))*pixsize  # peut être faire plus précis ensuite..
        if posx<np.amin(pos):
            posx = np.amin(pos)
        elif posx>np.amax(pos):
            posx = np.amax(pos)
        
        _Ar = fAr_lin(posx)
        _Ag = fAg_lin(posx)
        _Ab = fAb_lin(posx)
        _Br = fBr_lin(posx)
        _Bg = fBg_lin(posx)
        _Bb = fBb_lin(posx)
        corrArray[:,i,0] = _Ar + _Br*array[:,i,0]
        corrArray[:,i,1] = _Ag + _Bg*array[:,i,1]
        corrArray[:,i,2] = _Ab + _Bb*array[:,i,2]
    
    g.setArray(corrArray)
    doseImg_corr = g.convertToDose_cubicSplineFit(splineFile, 1500)
except ValueError as err:
    print('Erreur: '+err)

    
compare2Imgs(array[:,:,0], corrArray[:,:,0], 200, 140, plotwidth=450, 
             title1="no corr.", title2="corr.")